In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import json
import sqlite3
load_dotenv(override=True)

True

In [2]:
conn = sqlite3.connect('example.db')

In [3]:
model = ChatOpenAI(temperature=0)

In [4]:
# Define your desired data structure.
class Observation(BaseModel):
    title: str = Field(description="the title of the observation")
    description: str = Field(description="the detailed description of the observation")

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Observation)

In [5]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
chain = prompt | model | parser

In [6]:
data = chain.invoke({"query": "L'immagine mostra un gruppo di soldati a cavallo di cammelli, che sembrano essere in marcia o in parata. I soldati indossano uniformi militari e cappelli, e alcuni di loro tengono in mano delle armi. I cammelli sono adornati con coperte rosse decorate con frange dorate. La scena si svolge su una strada asfaltata, e sullo sfondo si intravede una leggera foschia che rende l'atmosfera un po' sfocata. La formazione dei soldati e dei cammelli è ordinata, con i cammelli disposti in fila."})

In [7]:
data

{'title': 'Soldiers on Camels',
 'description': 'A group of soldiers riding camels in what appears to be a march or parade. The soldiers are wearing military uniforms and hats, some of them holding weapons. The camels are adorned with red blankets decorated with golden fringes. The scene takes place on an asphalt road, with a slight haze in the background giving a slightly blurred atmosphere. The formation of soldiers and camels is orderly, with the camels lined up.'}

In [11]:
def create_tables():
    sql_statements = [ 
        """CREATE TABLE IF NOT EXISTS example_table (
                id INTEGER PRIMARY KEY AUTOINCREMENT, 
                title text NOT NULL, 
                description TEXT
        );"""
    ]
    try:
        with sqlite3.connect('example.db') as conn:
            cursor = conn.cursor()
            for statement in sql_statements:
                cursor.execute(statement)
            
            conn.commit()
    except sqlite3.Error as e:
        print(e)

In [12]:
create_tables()

In [13]:
conn.execute("INSERT INTO example_table (title, description) VALUES (?, ?)", 
                (data["title"], data["description"]))